In [1]:
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import plotly.graph_objects as go
from plotly.subplots import make_subplots

Want to predict which user have the most change to not re-subscribe at all 

User with Churn rate = 1 vs User with Churn Rate <1

Based on : 